In [1]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian
from sklearn.linear_model import LinearRegression
import addfips


In [2]:
states = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}


states = {v: k for k, v in states.items()}
states


{'Alaska': 'AK',
 'Alabama': 'AL',
 'Arkansas': 'AR',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'District of Columbia': 'DC',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Iowa': 'IA',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Massachusetts': 'MA',
 'Maryland': 'MD',
 'Maine': 'ME',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Missouri': 'MO',
 'Northern Mariana Islands': 'MP',
 'Mississippi': 'MS',
 'Montana': 'MT',
 'National': 'NA',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Nebraska': 'NE',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'Nevada': 'NV',
 'New York': 'NY',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 

In [3]:
def translate_ICD10(x, conversion):
    try:
        return conversion[x]
    except:
        return x
    
    



def clean_state_data(df):
    af = addfips.AddFIPS()
    try:
        df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])
    except: 
        df = df.dropna(subset=["State", "Population"])
    #df = df[df["Crude Rate"] != "Unreliable"]
    df.Population = df.Population.astype(float)
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    df["fips"] = df.apply(lambda x: af.get_state_fips(x["State"]), axis=1)
    df["Year"] = 2019
    df["State_Abv"] = df.apply(lambda x: states[x["State"]], axis=1)
    df = df.sort_values(by=["Rate"], ascending=False)
    df.drop(columns=["Notes", "State Code", ], inplace=True)
   
    try:
        f = open("data/Life/Deaths/ICD10Translation.json")
        translation = json.load(f)
        df["ICD-10 Common"] = df["ICD-10 113 Cause List"].apply(lambda x: translate_ICD10(x, translation))
        df.rename(columns={"ICD-10 Common":"Cause Of Death"}, inplace=True)
    except:
        None
    return df
    


def top_n(df, n):
    #Filter out low occuring chapters
    df = df.groupby("ICD-10 113 Cause List Code").filter(lambda x: len(x)>1)
    df = df.sort_values(by = ["Rate"], ascending = False)
    df = df.head(n).copy()
    top_codes = list(df["ICD-10 113 Cause List Code"].unique())
    df = df[df["ICD-10 113 Cause List Code"].isin(top_codes)]
    return df


firearms = pd.read_csv(r"data/Life/Deaths/FireArmState.txt", delimiter="	", na_values = ['Not Applicable'])
df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = pd.read_csv(r"data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
yearly_pop = pd.read_csv(r"data/Life/Deaths/2019State1YearPop.txt", delimiter="	", na_values = ['Not Applicable'])
state_deaths = pd.read_csv(r"data/Life/Deaths/2019StateDeaths.txt", delimiter="	", na_values = ['Not Applicable'])


df = clean_state_data(df)
firearms = clean_state_data(firearms)
state_pop = clean_state_data(state_pop)
yearly_pop = clean_state_data(yearly_pop)
state_deaths = clean_state_data(state_deaths)
state_pop["Total Pop"] = state_pop["Population"]

#top_df = top_n(df, 5)
#county = df.groupby(by=["2006 Urbanization Code", "ICD Sub-Chapter"]).mean().reset_index().sort_values(by=["Rate"], ascending=False)
state_pop.sample(5)
firearms = firearms.merge(state_pop[["State", "Total Pop"]], left_on="State",right_on="State")

In [4]:
state_deaths

,State,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv
142,Idaho,85+ years,85+,4511.0,29774.0,15150.8,15150.80,16,2019,ID
494,Utah,85+ years,85+,5798.0,38574.0,15030.8,15030.85,49,2019,UT
536,West Virginia,85+ years,85+,5969.0,40136.0,14871.9,14871.94,54,2019,WV
547,Wisconsin,85+ years,85+,19013.0,128309.0,14818.1,14818.13,55,2019,WI
218,Maine,85+ years,85+,4982.0,33800.0,14739.6,14739.64,23,2019,ME
...,...,...,...,...,...,...,...,...,...,...
409,Oregon,5-14 years,5-14,49.0,491536.0,10.0,9.97,41,2019,OR
46,California,5-14 years,5-14,490.0,5007987.0,9.8,9.78,06,2019,CA
80,Delaware,5-14 years,5-14,11.0,113975.0,Unreliable,9.65,10,2019,DE
232,Massachusetts,5-14 years,5-14,66.0,751035.0,8.8,8.79,25,2019,MA


In [5]:
df["age"] = df["Ten-Year Age Groups Code"].str.replace("+","").str.split("-").str[0].astype(int)
df

C:\Users\zeusg\AppData\Local\Temp/ipykernel_30752/3922431193.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["age"] = df["Ten-Year Age Groups Code"].str.replace("+","").str.split("-").str[0].astype(int)


,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Cause Of Death,age
10393,Michigan,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,13436.0,213136.0,6304.0,6303.96,26,2019,MI,Heart Disease (Major),85
16288,Ohio,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,14910.0,256272.0,5818.0,5818.04,39,2019,OH,Heart Disease (Major),85
182,Alabama,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,5294.0,91543.0,5783.1,5783.07,01,2019,AL,Heart Disease (Major),85
20777,Utah,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,2196.0,38574.0,5693.0,5692.95,49,2019,UT,Heart Disease (Major),85
1458,Arkansas,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,3409.0,59912.0,5690.0,5690.01,05,2019,AR,Heart Disease (Major),85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,California,Hodgkin disease (C81),GR113-038,35-44 years,35-44,10.0,5282100.0,Unreliable,0.19,06,2019,CA,Hodgkin disease (C81),35
2197,California,#Chronic lower respiratory diseases (J40-J47),GR113-082,15-24 years,15-24,10.0,5180973.0,Unreliable,0.19,06,2019,CA,Chronic lower respiratory diseases,15
1745,California,"Certain other intestinal infections (A04,A07-A09)",GR113-003,35-44 years,35-44,10.0,5282100.0,Unreliable,0.19,06,2019,CA,"Certain other intestinal infections (A04,A07-A09)",35
1873,California,Malignant neoplasms of corpus uteri and uterus...,GR113-031,25-34 years,25-34,11.0,6052952.0,Unreliable,0.18,06,2019,CA,Malignant neoplasms of corpus uteri and uterus...,25


In [ ]:
firearms.sample(3)

In [ ]:
yearly_pop.Population.sum()

In [ ]:
df["Ten-Year Age Groups"].unique()  

In [ ]:
teen = df[df["Ten-Year Age Groups"] == "15-24 years"]
teen.sample(3)

In [ ]:
teen

In [ ]:
teen_gun = teen[teen["Cause Of Death"] == "Homicide Guns"]
guns = df[df["Cause Of Death"] == "Homicide Guns"]


In [ ]:
fig  = go.Figure()
fig.add_trace(go.Bar(x = teen_gun["State"], y = teen_gun["Rate"]))

In [ ]:
guns = guns.sort_values(by="Rate", ascending=False)
px.bar(guns, x = "State", y="Rate", color="Ten-Year Age Groups")

In [ ]:
age_guns = guns.groupby(["Ten-Year Age Groups"]).mean().reset_index()
age_guns["Start Age"] = age_guns["Ten-Year Age Groups"].str.split("-").str[0].astype(int)
age_guns = age_guns.sort_values("Start Age")
age_guns

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=age_guns["Ten-Year Age Groups"], values=age_guns["Rate"], name="Guns"))


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=age_guns["Ten-Year Age Groups"], y=age_guns["Rate"], mode ="lines" ))

In [ ]:
guns
guns["Start Age"] = guns["Ten-Year Age Groups"].str.split("-").str[0].astype(int)
guns = guns.sort_values("Start Age")
guns.sample(3)

In [ ]:
fig = px.line(x = guns["Ten-Year Age Groups"], y= guns["Rate"], color = guns["State"])
fig

In [ ]:
firearms.sample(3)

In [ ]:
age_guns.sample(3)

In [ ]:
firearms

In [ ]:
firearms.rename(columns={"Single-Year Ages Code":"age"}, inplace=True)
firearms.dropna(subset=['age'], inplace=True)
firearms["age"] = firearms["age"].astype(int)
firearms = firearms.sort_values(by=["age","Rate"])


firearms_total = firearms.groupby(["age"]).mean().reset_index()

firearms_total.sample(10)

In [ ]:
firearms

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=firearms_total["age"], y=firearms_total["Rate"], mode ="lines" ))

In [ ]:
px.bar(firearms, x = "State", y="Rate", color="age", color_continuous_scale=px.colors.sequential.Bluered, title="Firearm Death Rate by Age Group", labels={"Rate":"Firearm Death Rate (per 100,000)"}, height=600)

In [ ]:
guns.sort_values(by="Rate", ascending=False, inplace=True)
px.bar(guns, x = "State", y="Rate", color="Ten-Year Age Groups Code", color_continuous_scale=px.colors.sequential.Bluered, title="Firearm Death Rate by Age Group", labels={"Rate":"Firearm Death Rate (per 100,000)"}, height=600)

In [ ]:
guns

In [6]:
part = Partisian.get_state_part_score()
final = pd.merge(part, firearms, on="fips")
final[final["State_x"] == "VT"]
state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")
state_totals["Rate"] = (state_totals["Deaths"]/state_totals["Total Pop"])*100000

C:\Users\zeusg\AppData\Local\Temp/ipykernel_30752/127021580.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Part_Score"], y=state_totals["Rate"], mode = "markers",  hovertext=state_totals["State_y"], marker=dict(size=state_totals["Total Pop"]/1000000, colorscale='bluered', color=state_totals["Part_Score"])))

In [ ]:
state_totals.sort_values("Rate", ascending=False, inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(x=state_totals["State_y"], y=state_totals["Rate"], name="Rate"'))
fig.show()

In [ ]:
state_totals[state_totals["State_y"] == "Vermont"]

In [ ]:
state_totals

In [ ]:
12/133915.0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Population"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["age"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [ ]:
teen_deaths = teen.groupby(by=["Cause Of Death"]).sum().reset_index()
teen_deaths.sort_values(by=["Deaths"], ascending=False, inplace=True)   

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=teen_deaths["Cause Of Death"], y=teen_deaths["Deaths"], name="Deaths"))

In [ ]:
yearly_pop

In [ ]:
usa_total = yearly_pop.groupby("Single-Year Ages").sum().reset_index()
usa_total["age"] = usa_total["Single-Year Ages"].str.replace("< 1 year","0").str.split(" ").str[0].astype(int)
usa_total.sort_values("age", inplace=True)
usa_total["Rate"] = (usa_total["Deaths"]/usa_total["Population"])*100000
usa_total

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Population"]))
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Rate"]))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa_total["age"], y=usa_total["Rate"], mode="lines"))

In [ ]:
df.sort_values(by="age", inplace=True)
df

In [ ]:
fig = px.scatter(df, x="age", y="Deaths", color="Cause Of Death")
fig.update_coloraxes(showscale=False)
fig.update_traces(showlegend=False)
fig.show()

In [ ]:
df["Ten-Year Age Groups Code"].unique()

In [7]:
child = state_deaths[state_deaths["Ten-Year Age Groups Code"] == '5-14']
part = Partisian.get_state_part_score()
child = pd.merge(child, part, left_on="State_Abv", right_on="State")


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=child['Part_Score'], y=child['Rate'],mode='markers',name='Child'))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=child["State_Abv"], y=child["Rate"], marker=dict(color=child["Part_Score"], colorscale="Bluered" ), name="Child"))

In [ ]:
child

In [ ]:
import GraphGenerator
def get_age_death(df, age):
    age_df = df[df["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    X = age_df['Part_Score'].values.reshape(-1,1)
    y = age_df['Rate']
    
    reg = LinearRegression().fit(X, y, sample_weight=age_df['Population'])
    x_range = np.linspace(X.min(), X.max())
    y = reg.predict(x_range.reshape(-1,1))
    
    fig = make_subplots(rows=1, cols=2, vertical_spacing= 0.01, horizontal_spacing= 0.01,  subplot_titles=("",""), column_widths=[0.8, 0.2], shared_yaxes=True)
    
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers', marker=dict(color=age_df["Part_Score"], colorscale="Bluered", size=age_df["Population"]/(age_df["Population"].max()/50)), name='Child', hovertext=age_df["State_x"]), row=1, col=1)
    
    hist_x = []
    for index, row in age_df.iterrows():
        hist_x.extend([row["Rate"] for i in range(int(round(row["Population"]/1000)))])
        row["Rate"]
    #fig.add_trace(go.Violin(y=hist_x, side="positive", name="Death Rate Distribution"), row=1, col=2)
    fig.add_trace(go.Violin(y=age_df['Rate'], side="positive",  name="Death Rate Distribution"), row=1, col=2)
    
    
    fig.add_trace(go.Scatter(x=x_range, y=y, mode='lines',  line=dict(color="green", dash="dash"), name='Best Fit Line'), row=1, col=1)
    
    fig.update_layout(title="Partisanship Score vs. Death Rate for " + age + " Year Olds", xaxis1_title="Partisanship Score", yaxis1_title="Death Rate (per 100,000)")
    fig.update_yaxes(showticklabels=False, row=1, col=2)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
    fig.update_layout(height=300,width=1200, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=25, b=0))
    
    fig.show()


In [ ]:
get_age_death(state_deaths,"85+")

In [ ]:
for age in state_deaths["Ten-Year Age Groups Code"].unique():
    None
    #get_age_death(state_deaths,age)

In [ ]:

fig = go.Figure()

for age in state_deaths["Ten-Year Age Groups Code"].unique():
    age_df = state_deaths[state_deaths["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers',name=age))
    #fig.add_trace(go.Bar(x=age_df["State_Abv"], y=age_df["Rate"], marker=dict(color=age_df["Part_Score"], colorscale="Bluered" ), name=age))

fig.show()

In [ ]:
heirarchy

In [ ]:
teen

In [8]:
teen = df[df["Ten-Year Age Groups Code"] == "85+"]
teen.sample(3)
teen_all = teen#[teen["ICD-10 Original"].str.contains("#") == True]
ny = teen_all[teen_all["State"] == "New York"]
ny.dropna(inplace=True, subset=["ICD-10 113 Cause List"])
ny.tail(3)

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Cause Of Death,age
14967,New York,"Bronchitis, chronic and unspecified (J40-J42)",GR113-083,85+ years,85+,14.0,460001.0,Unreliable,3.04,36,2019,NY,"Bronchitis, chronic and unspecified (J40-J42)",85
15153,New York,"Accidental exposure to smoke, fire and flames ...",GR113-121,85+ years,85+,13.0,460001.0,Unreliable,2.83,36,2019,NY,"Accidental exposure to smoke, fire and flames ...",85
14875,New York,Acute and subacute endocarditis (I33),GR113-065,85+ years,85+,10.0,460001.0,Unreliable,2.17,36,2019,NY,Acute and subacute endocarditis (I33),85


In [14]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [15]:
#ny.loc[15081]

In [31]:
ny["ICD-10 113 Cause List"] = pd.Categorical(ny["ICD-10 113 Cause List"], categories = list(heirarchy.keys()))
ny = ny.sort_values(by=["ICD-10 113 Cause List"])
ny["ICD-10 113 Cause List"].unique()
ny.dropna(subset=["ICD-10 113 Cause List"], inplace=True)

In [32]:
ny

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Cause Of Death,age
14572,New York,"Certain other intestinal infections (A04,A07-A09)",GR113-003,85+ years,85+,176.0,460001.0,38.3,38.26,36,2019,NY,"Certain other intestinal infections (A04,A07-A09)",85
14581,New York,#Septicemia (A40-A41),GR113-010,85+ years,85+,559.0,460001.0,121.5,121.52,36,2019,NY,Septicemia,85
14600,New York,Other and unspecified infectious and parasitic...,GR113-018,85+ years,85+,196.0,460001.0,42.6,42.61,36,2019,NY,Other and unspecified infectious and parasitic...,85
14610,New York,#Malignant neoplasms (C00-C97),GR113-019,85+ years,85+,6597.0,460001.0,1434.1,1434.13,36,2019,NY,Cancer,85
14759,New York,"#In situ neoplasms, benign neoplasms and neopl...",GR113-044,85+ years,85+,318.0,460001.0,69.1,69.13,36,2019,NY,"#In situ neoplasms, benign neoplasms and neopl...",85
14765,New York,#Anemias (D50-D64),GR113-045,85+ years,85+,94.0,460001.0,20.4,20.43,36,2019,NY,#Anemias (D50-D64),85
14772,New York,#Diabetes mellitus (E10-E14),GR113-046,85+ years,85+,1153.0,460001.0,250.7,250.65,36,2019,NY,Diabetes,85
14776,New York,#Nutritional deficiencies (E40-E64),GR113-047,85+ years,85+,167.0,460001.0,36.3,36.30,36,2019,NY,#Nutritional deficiencies (E40-E64),85
14783,New York,#Parkinson disease (G20-G21),GR113-051,85+ years,85+,733.0,460001.0,159.3,159.35,36,2019,NY,#Parkinson disease (G20-G21),85
14787,New York,#Alzheimer disease (G30),GR113-052,85+ years,85+,2680.0,460001.0,582.6,582.61,36,2019,NY,Alzheimer,85


In [58]:
labels = ["All"]
labels.extend(list(ny["ICD-10 113 Cause List"]))

values = [0]
values.extend(list(ny.Rate))


parents = []
for i,label in enumerate(labels):
    if label == "All":
        parents.append("")
    else:
        try:
            parents = flat_heirarchy[label]["parents"]
            if len(parent) > 0:
                parents.append(parent[0])
            else:
                parents.append("")
                #values[i] = values[i]-values[parents.index(heirarchy[label])]
            
        except:
            parents.append("")  



In [806]:
test_labels = labels[:11]
test_parents = parents[:11]
test_values = values[:11]
test_values[0] = sum(test_values[1:])

In [807]:
labels

['All',
 'Certain other intestinal infections (A04,A07-A09)',
 '#Septicemia (A40-A41)',
 'Other and unspecified infectious and parasitic diseases and their sequelae (A00,A05,A20-A36,A42-A44,A48-A49,A54-A79,A81-A82,A85.0-A85.1,A85.8,A86-B04,B06-B09,B25-B49,B55-B99,U07.1)',
 '#Malignant neoplasms (C00-C97)',
 '#In situ neoplasms, benign neoplasms and neoplasms of uncertain or unknown behavior (D00-D48)',
 '#Anemias (D50-D64)',
 '#Diabetes mellitus (E10-E14)',
 '#Nutritional deficiencies (E40-E64)',
 '#Parkinson disease (G20-G21)',
 '#Alzheimer disease (G30)',
 'Major cardiovascular diseases (I00-I78)',
 'Other disorders of circulatory system (I80-I99)',
 '#Influenza and pneumonia (J09-J18)',
 '#Chronic lower respiratory diseases (J40-J47)',
 '#Pneumonitis due to solids and liquids (J69)',
 'Other diseases of respiratory system (J00-J06,J30- J39,J67,J70-J98)',
 '#Peptic ulcer (K25-K28)',
 '#Chronic liver disease and cirrhosis (K70,K73-K74)',
 '#Cholelithiasis and other disorders of gallbl

In [808]:
parents

['', '']

In [809]:
test_values

[50.3400000000001,
 1.69,
 0.24,
 0.56,
 10.69,
 6.2,
 2.57,
 5.4,
 0.020000000000102602,
 9.45,
 13.52]

In [810]:
fig =go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
))

fig.show()

In [811]:
flat_heirarchy["#Human immunodeficiency virus (HIV) disease (B20-B24)"]

{'common_name': None,
 'children': [],
 'parents': [],
 'ranges': {'H': [], 'B': [20, 21, 22, 23, 24]},
 'chapters': ['B'],
 'base': True,
 'Rate': 2.57}

In [812]:
df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])

In [813]:
state

,Notes,State,State Code,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate
1748,NaN,California,6.0,"Certain other intestinal infections (A04,A07-A09)",GR113-003,65-74 years,65-74,178.0,3386670.0,5.3,5.26
1753,NaN,California,6.0,#Tuberculosis (A16-A19),GR113-004,65-74 years,65-74,27.0,3386670.0,0.8,0.80
1758,NaN,California,6.0,Respiratory tuberculosis (A16),GR113-005,65-74 years,65-74,19.0,3386670.0,Unreliable,0.56
1766,NaN,California,6.0,#Septicemia (A40-A41),GR113-010,65-74 years,65-74,362.0,3386670.0,10.7,10.69
1771,NaN,California,6.0,#Viral hepatitis (B15-B19),GR113-015,65-74 years,65-74,210.0,3386670.0,6.2,6.20
...,...,...,...,...,...,...,...,...,...,...,...
2493,NaN,California,6.0,Assault (homicide) by other and unspecified me...,GR113-129,65-74 years,65-74,45.0,3386670.0,1.3,1.33
2505,NaN,California,6.0,"Events of undetermined intent (Y10-Y34,Y87.2,Y...",GR113-131,65-74 years,65-74,28.0,3386670.0,0.8,0.83
2512,NaN,California,6.0,Other and unspecified events of undetermined i...,GR113-133,65-74 years,65-74,27.0,3386670.0,0.8,0.80
2516,NaN,California,6.0,#Complications of medical and surgical care (Y...,GR113-135,65-74 years,65-74,74.0,3386670.0,2.2,2.19


In [814]:
state["Rate"]

1748     5.26
1753     0.80
1758     0.56
1766    10.69
1771     6.20
        ...  
2493     1.33
2505     0.83
2512     0.80
2516     2.19
2521     3.57
Name: Rate, Length: 106, dtype: float64

In [815]:
df["Ten-Year Age Groups Code"].unique()

array(['1', '55-64', '65-74', '75-84', '85+', '25-34', '35-44', '45-54',
       '5-14', '1-4', '15-24'], dtype=object)

In [816]:
age = '45-54'

In [817]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [122]:
def load_state_cod(state, age_group = '75-84'):
    df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    state_df= df[df["State"] == state]
    state_df = state_df[state_df["Ten-Year Age Groups Code"] =='75-84']
    state_df["Rate"] = (state_df["Deaths"] / state_df["Population"])*100000
    state_df["Rate"] = np.round(state_df["Rate"], 2)
    state_df["Child_Adj_Rate"] = state_df["Rate"]
    
    state_df["Parent"] = ""
    state_df["Children"] = ""
    state_df["Common Name"] = ""
    
    
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)

    targets =[]
    sources = []
    values = []

    sources.append("")
    targets.append("All")
    values.append(0)
    values.extend(list(state_df.Rate))


    for i, row in state_df.iterrows():
        try:
            structure = flat_heirarchy[row["ICD-10 113 Cause List"]]
            state_df.at[i, "Common Name"] = structure["common_name"]
            targets.append(structure["common_name"])
            structure["Rate"] = row["Rate"]
            
            if len(structure["parents"])  == 1:
                parent = flat_heirarchy[structure["parents"][0]]["common_name"]
                state_df.loc[i, "Parent"] = parent
                sources.append(parent)
            else:
                state_df.loc[i, "Parent"] = "All"
                sources.append("All")
        except Exception as e:
            print(e)
        if len(structure["children"]) > 0:
            state_df.at[i, "Children"] = structure["children"]
             
    for i, source in enumerate(sources):
        if source == "":
            None
        else:
            try:
                parent_index = targets.index(source)
                values[parent_index] = max( values[parent_index] - values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[parent_index]
                
            except:
                values[i] = max(values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[i]
                
    size = 99
    plot_dict = {"values":values[:size], "targets":targets[:size], "parents":sources[:size]}
    return state_df, plot_dict

In [123]:
df, plot_dic = load_state_cod("California")

In [125]:
plot_dic

{'values': [0,
  3.0500000000000007,
  0.41000000000000014,
  1.18,
  23.74,
  3.64,
  1.94,
  11.05,
  0.020000000000038654,
  13.87,
  20.8,
  21.1,
  71.87,
  45.78,
  78.63,
  4.82,
  204.92,
  12.46,
  52.77,
  3.35,
  17.34,
  20.86,
  67.76,
  19.92,
  31.26,
  20.22,
  0.1700000000000088,
  1.53,
  37.91,
  38.79,
  23.86,
  112.01,
  23.92,
  5.17,
  151.39,
  0.23000000000000043,
  10.17,
  0.65,
  90.62,
  234.13,
  0,
  0,
  5.23,
  60.59,
  13.11,
  0,
  143.98,
  2.29,
  373.35,
  83.63,
  289.73,
  106.9,
  1.53,
  0.71,
  106.9,
  138.16,
  79.69,
  247.77,
  8.46,
  0,
  11.64,
  12.4,
  3.23,
  0,
  5.35,
  77.46,
  0.47999999999996135,
  10.34,
  3.58,
  223.08,
  0.76,
  23.04,
  59.18,
  7.52,
  0.71,
  3.88,
  0,
  13.93,
  23.98,
  6.88,
  0.4099999999999966,
  0.59,
  60.53,
  1.7,
  1.12,
  2.7,
  5.64,
  327.28,
  0,
  0,
  15.04,
  0.59,
  1.0,
  10.580000000000005,
  43.66,
  1.82,
  1.35,
  3.94,
  10.4],
 'targets': ['All',
  'Certain other intestinal infe

In [126]:
plot_dic["targets"][:10]

['All',
 'Certain other intestinal infections',
 '#Tuberculosis',
 'Respiratory tuberculosis',
 'Septicemia',
 '#Viral hepatitis',
 'HIV',
 'Other and unspecified infectious and parasitic diseases and their sequelae',
 'Cancer',
 'Malignant neoplasms of lip, oral cavity and pharynx']

In [127]:
plot_dic["parents"][:10]

['',
 'All',
 'All',
 '#Tuberculosis',
 'All',
 'All',
 'All',
 'All',
 'All',
 'Cancer']

In [129]:
fig = go.Figure(go.Treemap(
    labels = plot_dic["targets"],
    parents =  plot_dic["parents"],
    values=plot_dic['values'],
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [76]:
fig = px.sunburst(
    plot_dic,
    names='targets',
    parents='parents',
    values='values',
)
fig.update_layout(
    autosize=False,
    width=1800,
    height=1000,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)
fig.show()

fig.write_html("CauseOfCircle.html")

In [77]:
fig = px.treemap(
    plot_dic,
    names='targets',
    parents='parents',
    values='values',
)

fig.update_layout(
    autosize=False,
    width=1800,
    height=1000,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()
fig.write_html("CauseOfDeathTest.html")

In [78]:
plot_dict.keys()

dict_keys(['values', 'targets', 'parents'])

In [686]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["A1", "A2", "B1", "B2", "C1", "C2"],
      color = "blue"
    ),
    link = dict(
      source = [0, 1, 0, 2, 3, 3], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [2, 3, 3, 4, 4, 5],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [137]:
print(f"source: {character} \n  target: {parent}  \n  value: {t_value}")

print(f"source: {len(character)} \n  target: {len(parent)}  \n  value: {len(t_value)}")

source: (['Eve', 'Cain', 'Seth', 'Enos', 'Noam', 'Abel', 'Awan', 'Enoch', 'Azura'],) 
  target: (['', 'Eve', 'Eve', 'Seth', 'Seth', 'Eve', 'Eve', 'Awan', 'Eve'],)  
  value: [5.255900338680769e-05, 7.972433097998919e-06, 5.610230698591832e-06, 0.0001068896585731707, 6.200781298443604e-05, 2.5688951093552072e-05, 5.403537988643712e-05, 0.004667121390628553, 9.448809597628349e-05, 0.00013523608736605575]
source: 1 
  target: 1  
  value: 10


In [134]:
fig =go.Figure(go.Sunburst(
    labels=t_source,
    parents=t_target,
    values=t_value,
))

fig.show()

In [122]:
target

['Certain other intestinal infections (A04,A07-A09)',
 '#Tuberculosis (A16-A19)',
 'Respiratory tuberculosis (A16)',
 '#Septicemia (A40-A41)',
 '#Viral hepatitis (B15-B19)',
 '#Human immunodeficiency virus (HIV) disease (B20-B24)',
 'Other and unspecified infectious and parasitic diseases and their sequelae (A00,A05,A20-A36,A42-A44,A48-A49,A54-A79,A81-A82,A85.0-A85.1,A85.8,A86-B04,B06-B09,B25-B49,B55-B99,U07.1)',
 '#Malignant neoplasms (C00-C97)',
 'Malignant neoplasms of lip, oral cavity and pharynx (C00-C14)',
 'Malignant neoplasm of esophagus (C15)',
 'Malignant neoplasm of stomach (C16)',
 'Malignant neoplasms of colon, rectum and anus (C18-C21)',
 'Malignant neoplasms of liver and intrahepatic bile ducts (C22)',
 'Malignant neoplasm of pancreas (C25)',
 'Malignant neoplasm of larynx (C32)',
 'Malignant neoplasms of trachea, bronchus and lung (C33-C34)',
 'Malignant melanoma of skin (C43)',
 'Malignant neoplasm of breast (C50)',
 'Malignant neoplasm of cervix uteri (C53)',
 'Malign

In [120]:
len(source)
#len(target)
#len(value)

106

In [ ]:
source